Clone Repo

In [ ]:
!git clone https://github.com/biomed-AI/GraphBepi
!mv GraphBepi/* .

install/import required libraries

`!pip install torchmetrics pytorch_lightning tool fair-esm`

`!pip install "fair-esm[esmfold]"`

`!pip install 'dllogger @ git+https://github.com/NVIDIA/dllogger.git'`

`!pip install 'openfold @ git+https://github.com/aqlaboratory/openfold.git@4b41059694619831a7db195b7e0988fc4ff3a307'`

`!pip3 install lightning`

`!pip3 install tensorflow --user`

`!pip3 install omegaconf`

In [ ]:
import os
import time
import torch
import random
import warnings;
import argparse
import numpy as np
import pandas as pd
import pickle as pk
import torch.nn as nn
import torchmetrics as tm
import pytorch_lightning as pl
import torch.nn.functional as F
from tool import METRICS
from tqdm import tqdm,trange
from model import GraphBepi
from dataset import PDB,collate_fn,chain
from collections import defaultdict
from torch.utils.data import DataLoader,Dataset
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import Callback,EarlyStopping, ModelCheckpoint
warnings.simplefilter('ignore')
import tensorflow as tf

Set seed and device

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def seed_everything(seed=2022):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(2022)

Download dataset

In [5]:
!python dataset.py --gpu 0 --root data/BCE_633

mkdir: cannot create directory ‘PDB’: File exists
mkdir: cannot create directory ‘purePDB’: File exists
mkdir: cannot create directory ‘feat’: File exists
mkdir: cannot create directory ‘dssp’: File exists
mkdir: cannot create directory ‘graph’: File exists
100%|█████████████████████████| 633/633 [29:13<00:00,  2.77s/it, protein=3lh2_V]


Train GraphBepi

In [21]:
!python train.py --dataset BCE_633 --epochs 50

100%|███████████████████████████| 577/577 [00:07<00:00, 77.00it/s, chain=3lh2_V]
0it [00:00, ?it/s]
100%|█████████████████████████████| 56/56 [00:00<00:00, 69.06it/s, chain=7ue9_C]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params
-------------------------------------------
0 | loss_fn     | BCELoss    | 0     
1 | W_v         | Linear     | 655 K 
2 | W_u1        | AE         | 69.9 K
3 | edge_linear | Sequential | 3.3 K 
4 | gat         | EGAT       | 8.5 M 
5 | lstm1       | LSTM       | 1.2 M 
6 | lstm2       | LSTM       | 1.2 M 
7 | mlp         | Sequential | 262 K 
-------------------------------------------
11.9 M    Trainable params
0         Non-trainable params
11.9 M    Total params
47.536    Total estimated model params size (MB)
Epoch 0:  91%|▉| 144/158 [00:45<00:04,  3.17it/s, loss=0.384, v_

Download protein FASTA

In [4]:
!wget https://rest.uniprot.org/uniprotkb/P56856.fasta

--2023-01-13 05:23:45--  https://rest.uniprot.org/uniprotkb/P56856.fasta
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344 [text/plain]
Saving to: ‘P56856.fasta’

P56856.fasta        100%[===================>]     344  --.-KB/s    in 0s      

2023-01-13 05:23:46 (13.5 MB/s) - ‘P56856.fasta’ saved [344/344]



ESMFold for FASTA - PDB and GraphBepi on the PDB file

In [4]:
!python test.py -i P56856.fasta -f --gpu 0 -o ./output

Preparing...
Running ESMfold...
100%|█████████████████████████████████████████████| 3/3 [00:04<00:00,  1.50s/it]
Processing...
100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  3.53it/s]
Predicting...
100%|█| 3/3 [00:00<00:00, 193.77it/s, chain=GGLTLIGGVMMCIACRGLAPEETNYKAVSYHASGHS
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Testing DataLoader 0: 100%|███████████████████████| 1/1 [00:00<00:00, 33.94it/s]
Fin
